<a href="https://colab.research.google.com/github/Sunn2x333/scalar_framework/blob/main/Beam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

# Original parameters
Phi_amp = 1e-6
k_phi = 0.008 / (Phi_amp**2 * (np.pi/1)**2 / 2)
k_m = 0.1
B_grad = 5.0
B_crit = 1e9
tau_beam = 888.0

def compute_delta_lambda(L, B_grad=5.0):
    """Original distortion model"""
    if L == 0:
        return 0
    grad_sq = (np.pi**2 * Phi_amp**2) / (2 * L**2)
    delta_grad = k_phi * grad_sq
    delta_magnetic = k_m * (B_grad**2 / B_crit**2)
    return delta_grad + delta_magnetic

def compute_tau(delta, tau_0=tau_beam):
    """Lifetime with distortion: τ = τ₀/(1+δλ)"""
    return tau_0 / (1 + delta)

def beam_with_induced_wiggling(exposure_factor, L_ref=0.7, tau_0=888.0):
    """
    Simulate beam with artificial wiggling via exposure factor
    Effective δλ_beam = δλ_trap × (exposure_time_beam / exposure_time_trap) × f
    """
    # Reference trap distortion
    delta_trap = compute_delta_lambda(L_ref, B_grad)

    # Nominal beam exposure vs trap exposure
    t_beam_nominal = 5e-4  # 0.5 ms transit time
    t_trap = tau_0         # ~888s full exposure

    # Effective distortion in "messed" beam
    exposure_ratio = t_beam_nominal / t_trap
    delta_beam = delta_trap * exposure_factor * exposure_ratio

    tau_beam_mod = tau_0 / (1 + delta_beam)
    return delta_beam, tau_beam_mod

# Simulate
exposure_factors = np.logspace(0, 7, 8)
L_trap = 0.7

print("Exposure Factor | Effective δλ | Apparent τ_n (s)")
print("-" * 45)

for f in exposure_factors:
    delta, tau = beam_with_induced_wiggling(f, L_trap)
    print(f"{f:>12.0e} | {delta:>12.4f} | {tau:>14.1f}")

# Trap reference
delta_ucn = compute_delta_lambda(L_trap)
tau_ucn = compute_tau(delta_ucn)
print(f"\nUCNτ trap (L={L_trap}m): δλ={delta_ucn:.4f}, τ={tau_ucn:.1f} s")

# Find exposure factor needed to match trap effect
f_needed = (delta_ucn / compute_delta_lambda(L_trap)) / (5e-4 / 888)
print(f"\nExposure factor needed to match trap δλ: {f_needed:.1e}")



Exposure Factor | Effective δλ | Apparent τ_n (s)
---------------------------------------------
       1e+00 |       0.0000 |          888.0
       1e+01 |       0.0000 |          888.0
       1e+02 |       0.0000 |          888.0
       1e+03 |       0.0000 |          888.0
       1e+04 |       0.0001 |          887.9
       1e+05 |       0.0009 |          887.2
       1e+06 |       0.0092 |          879.9
       1e+07 |       0.0919 |          813.2

UCNτ trap (L=0.7m): δλ=0.0163, τ=873.7 s

Exposure factor needed to match trap δλ: 1.8e+06
